In [ ]:
!pip install tensorflow

In [ ]:
# @title 1. Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set seeds for reproducibility (Crucial for scientific reporting)
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

print("Libraries loaded successfully.")

In [ ]:
# @title 2. Load Data and Initial Preprocessing
import pandas as pd
import os

# Define the data directory relative to this script
data_dir = './data'
# Update the filename to the .zip version
file_name = 'asv_interpretability_dataset_modified.zip'
file_path = os.path.join(data_dir, file_name)

# Check if file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Data file not found at {file_path}. Please ensure the 'data' folder contains the zipped dataset.")

# Load the data directly from the zip file
# Pandas automatically detects the zip compression
df = pd.read_csv(file_path, dtype={'PatientID': str})
print(f"Successfully loaded data from {file_path}")

# --- Helper: Handle NeutrophilCount with '<0.1' values ---
def parse_neutrophil(value):
    try:
        return float(value)
    except:
        if isinstance(value, str) and "<" in value:
            threshold = float(value.replace("<", "").strip())
            return threshold / 2
        return np.nan

# --- Helper: Create Proxy Labels (Clinical Dysbiosis) ---
def label_dysbiosis(row):
    # Proxy definition: High Temp + Low Neutrophils + Liquid Stool
    is_temp_abnormal = row['MaxTemperature'] > 38.0
    is_neutro_low = row['NeutrophilCount'] < 500
    is_consistency_liquid = row.get('Consistency_liquid', 0) == 1
    return int(is_temp_abnormal and is_neutro_low and is_consistency_liquid)

# 1. Clean Neutrophils
df['NeutrophilCount'] = df['NeutrophilCount'].apply(parse_neutrophil)
# Impute missing neutrophils with median (optional, depends on your specific logic)
# df['NeutrophilCount'].fillna(df['NeutrophilCount'].median(), inplace=True)

# 2. One-hot encode stool consistency
df = pd.get_dummies(df, columns=['Consistency'])

# 3. Log transform Genus-relative abundances (Compositional handling)
# Ensure no negative values or zeros break the log
df['RelativeAbundance'] = df['RelativeAbundance'].astype(float)
df['RelativeAbundance'] = np.log1p(df['RelativeAbundance'])

# 4. Generate Labels (Row-wise)
df['DysbiosisLabel'] = df.apply(label_dysbiosis, axis=1)

# 5. Pivot to Wide Format (Time Series Format)
# Keep metadata
metadata_cols = ['PatientID', 'SampleID', 'DayRelativeToNearestHCT',
                 'MaxTemperature', 'NeutrophilCount'] + \
                [col for col in df.columns if col.startswith('Consistency_')] + \
                ['DysbiosisLabel']

# Pivot Genus
genus_pivot = df.pivot_table(index=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'],
                             columns='Genus', values='RelativeAbundance', fill_value=0).reset_index()

# Merge Metadata back
metadata = df[metadata_cols].drop_duplicates(subset=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'])
merged_df = pd.merge(genus_pivot, metadata, on=['PatientID', 'SampleID', 'DayRelativeToNearestHCT'], how='left')

print(f"Data Processed. Total Samples: {len(merged_df)}")
print(f"Total Unique Patients: {merged_df['PatientID'].nunique()}")

In [ ]:
# @title 3. Patient-Level Splitting & Scaling (PREVENT DATA LEAKAGE)

# --- STEP A: Split Patients First ---
# We split the Patient IDs, NOT the sequences.
unique_patients = merged_df['PatientID'].unique()
np.random.shuffle(unique_patients) # Randomize patient order

n_total = len(unique_patients)
n_train = int(0.70 * n_total)
n_val = int(0.15 * n_total)

train_pids = unique_patients[:n_train]
val_pids = unique_patients[n_train : n_train + n_val]
test_pids = unique_patients[n_train + n_val:]

print(f"Patients in Train: {len(train_pids)}")
print(f"Patients in Val:   {len(val_pids)}")
print(f"Patients in Test:  {len(test_pids)}")

# Create separate DataFrames based on Patient ID
df_train = merged_df[merged_df['PatientID'].isin(train_pids)].copy()
df_val = merged_df[merged_df['PatientID'].isin(val_pids)].copy()
df_test = merged_df[merged_df['PatientID'].isin(test_pids)].copy()

# --- STEP B: Feature Selection (Train Only) ---
# Identify genus columns
all_genus_cols = genus_pivot.columns.drop(['PatientID', 'SampleID', 'DayRelativeToNearestHCT']).tolist()

# Calculate variance ONLY on Training data to avoid leakage
train_variances = df_train[all_genus_cols].var()
# Drop columns with near-zero variance in training set
non_zero_var_cols = train_variances[train_variances > 1e-6].index.tolist()

# Define final feature list (Microbiome + Stool Consistency)
# Exclude Temp/Neutrophils from Input X (as they define the label Y)

# Changed on 02 December, 2025
# feature_cols = non_zero_var_cols + [col for col in merged_df.columns if 'Consistency' in col]

# === CRITICAL FIX ===
# Input Features = Microbiome ONLY.
# We REMOVE 'Consistency' because it is part of the Label definition (Leakage).
feature_cols = non_zero_var_cols
# feature_cols += [col for col in merged_df.columns if 'Consistency' in col] <--- REMOVED THIS LINE

print(f"Selected {len(feature_cols)} features (Microbiome Genus Only).")

#print(f"Selected {len(feature_cols)} features based on Training Set variance.")

# --- STEP C: Scaling (Fit on Train Only) ---
scaler = MinMaxScaler()

# 1. FIT and TRANSFORM on Training
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])

# 2. TRANSFORM Only on Val/Test (using Train statistics)
df_val[feature_cols] = scaler.transform(df_val[feature_cols])
df_test[feature_cols] = scaler.transform(df_test[feature_cols])

print("Scaling complete. Data leakage prevented.")

In [ ]:
# @title 4. Sequence Generation (Sliding Window)

def build_sequences(df, feature_cols, label_col='DysbiosisLabel', seq_len=14):
    """
    Generates sequences strictly within patient groups.
    """
    X_sequences = []
    y_labels = []

    # Group by patient to ensure window never crosses patient boundaries
    for pid, group in df.groupby('PatientID'):
        # Sort by time
        group = group.sort_values('DayRelativeToNearestHCT')

        values = group[feature_cols].values
        labels = group[label_col].values

        # Sliding window
        if len(values) >= seq_len:
            for i in range(len(values) - seq_len + 1):
                seq = values[i:i+seq_len]
                label_window = labels[i:i+seq_len]

                # Label Logic: If ANY point in window is dysbiotic, label=1
                # (Or use label_window[-1] for "next step prediction")
                label = int(label_window.max())

                X_sequences.append(seq)
                y_labels.append(label)

    return np.array(X_sequences), np.array(y_labels)

# Build sequences for each split independently
SEQ_LEN = 14

X_train, y_train = build_sequences(df_train, feature_cols, seq_len=SEQ_LEN)
X_val, y_val = build_sequences(df_val, feature_cols, seq_len=SEQ_LEN)
X_test, y_test = build_sequences(df_test, feature_cols, seq_len=SEQ_LEN)

print(f"Training Sequences: {X_train.shape}")
print(f"Validation Sequences: {X_val.shape}")
print(f"Testing Sequences: {X_test.shape}")

In [ ]:
import tensorflow as tf
import os

# Define the path relative to the 'models' folder
model_path = "./models/DynaBiome_PatientSplit_Model.keras"

# Check if file exists (Good practice for public code)
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}. Did you upload it to the 'models' folder?")

# Load the trained model
autoencoder = tf.keras.models.load_model(model_path)
print(f"Model loaded successfully from {model_path}")

In [ ]:
# @title 6. Generate Reconstruction Errors (Features)

import numpy as np

def get_mae_features(model, X_data):
    """
    Passes data through the frozen Autoencoder and calculates MAE.
    Returns a vector of shape (n_samples, ).
    """
    # 1. Reconstruct sequences
    reconstructions = model.predict(X_data, verbose=0)

    # 2. Calculate Mean Absolute Error (averaged over time and features)
    # Axis 1 = Timesteps, Axis 2 = Features
    mae_loss = np.mean(np.abs(X_data - reconstructions), axis=(1, 2))
    return mae_loss

print("Generating features using the frozen LSTM Autoencoder...")

# 1. Generate Errors for the TRAINING set (Both Normal and Dysbiotic)

train_mae = get_mae_features(autoencoder, X_train)

# 2. Generate Errors for the VALIDATION set

val_mae = get_mae_features(autoencoder, X_val)

# 3. Generate Errors for the TEST set

test_mae = get_mae_features(autoencoder, X_test)

# Reshape for Scikit-Learn (Must be 2D array: n_samples x n_features)
X_train_feat = train_mae.reshape(-1, 1)
X_val_feat   = val_mae.reshape(-1, 1)
X_test_feat  = test_mae.reshape(-1, 1)

print(f"Feature Extraction Complete.")
print(f"Training Features Shape:   {X_train_feat.shape} (Labels: {len(y_train)})")
print(f"Validation Features Shape: {X_val_feat.shape}   (Labels: {len(y_val)})")
print(f"Test Features Shape:       {X_test_feat.shape}  (Labels: {len(y_test)})")

In [ ]:
!pip install statsmodels
!pip install xgboost

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, average_precision_score, accuracy_score
import numpy as np
from sklearn.metrics import precision_recall_curve # Added for ensemble F1 tuning

# --- 1. DATA PREPARATION ---
# Ensure features are 2D arrays (N_samples, 1)
X_train_feat = train_mae.reshape(-1, 1)
X_val_feat   = val_mae.reshape(-1, 1)
X_test_feat  = test_mae.reshape(-1, 1)

print(f"Training Data Shape: {X_train_feat.shape}")

# Define list of ensemble names for later iteration
ensemble_names_list = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']

# --- 2. DEFINE PARAMETER GRIDS ---
param_grids = {
    "Logistic Regression": {
        'model': LogisticRegression(random_state=42, solver='liblinear'),
        'params': {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}
    },
    "KNN": {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [5, 20, 50], 'weights': ['uniform', 'distance']}
    },
    "Random Forest": {
        'model': RandomForestClassifier(random_state=42),
        'params': {'n_estimators': [100, 200], 'max_depth': [3, 5, 10], 'min_samples_split': [5, 10]}
    },
    "XGBoost": {
        'model': XGBClassifier(eval_metric='logloss', random_state=42),
        'params': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]}
    },
    "MLP": {
        'model': MLPClassifier(max_iter=500, random_state=42),
        'params': {'hidden_layer_sizes': [(32, 16), (64, 32)], 'activation': ['relu'], 'alpha': [0.001, 0.01]}
    }
}

best_models = {}

print("--- STARTING HYPERPARAMETER TUNING (GridSearchCV) ---")
print("Note: Tuning is performed on Training Set with 5-Fold CV.\n")

# --- 3. SUPERVISED MODELS LOOP ---
for name, config in param_grids.items():
    print(f"Tuning {name}...")
    grid = GridSearchCV(estimator=config['model'], param_grid=config['params'], cv=5, scoring='roc_auc', n_jobs=-1)
    grid.fit(X_train_feat, y_train)
    best_models[name] = grid.best_estimator_
    print(f"  Best CV AUC: {grid.best_score_:.4f}")

# --- 4. ONE-CLASS SVM SPECIAL HANDLING ---
print("\nTuning One-Class SVM (Special Unsupervised Handling)...")
best_ocsvm = None
best_ocsvm_f1 = -1
X_train_normal_feat = X_train_feat[y_train == 0]

for nu in [0.01, 0.05, 0.1, 0.2, 0.3]:
    ocsvm = OneClassSVM(kernel='rbf', gamma='scale', nu=nu)
    ocsvm.fit(X_train_normal_feat)
    preds = ocsvm.predict(X_val_feat)
    binary_preds = np.where(preds == -1, 1, 0)
    f1 = f1_score(y_val, binary_preds)
    if f1 > best_ocsvm_f1:
        best_ocsvm_f1 = f1
        best_ocsvm = ocsvm

best_models["One-Class SVM"] = best_ocsvm
print(f"  Best OCSVM Val F1: {best_ocsvm_f1:.4f}")

# --- 5. COLLECT PREDICTIONS FOR ENSEMBLING & TUNING THRESHOLDS ---
# We need Validation Predictions (to train Meta-Learners and tune F1 thresholds)
# and Test Predictions (for Final Evaluation)
val_probs_dict = {}
test_probs_dict = {}
val_aucs = {} # For Weighted Ensemble
best_thresholds = {}

print("\n--- Generating Predictions for Ensembling & Tuning Thresholds ---")

for name, clf in best_models.items():
    if name == "One-Class SVM": continue # OCSVM excluded from standard ensembles for now

    # Predict on Validation
    val_probs = clf.predict_proba(X_val_feat)[:, 1]
    val_probs_dict[name] = val_probs

    # Store Val AUC for weighting
    val_aucs[name] = roc_auc_score(y_val, val_probs)

    # Threshold Tuning for Individual Models (Youden's Index for ROC)
    fpr, tpr, thresholds = roc_curve(y_val, val_probs)
    optimal_idx = np.argmax(tpr - fpr)
    # Ensure optimal_idx is within bounds of thresholds array
    if optimal_idx < len(thresholds):
        best_thresholds[name] = thresholds[optimal_idx]
    else:
        best_thresholds[name] = 0.5 # Default if threshold array is empty/problematic

    # Predict on Test
    test_probs = clf.predict_proba(X_test_feat)[:, 1]
    test_probs_dict[name] = test_probs

# ==========================================================
# --- 6. ADVANCED ENSEMBLE METHODS (CALCULATE PROBABILITIES) ---
# ==========================================================

# A. Averaged Ensemble - Calculate Test Probs
avg_probs = np.mean(list(test_probs_dict.values()), axis=0)
test_probs_dict['Averaged Ensemble'] = avg_probs # Store test probs

# B. Weighted Ensemble - Calculate Test Probs
total_auc = sum(val_aucs.values())
weights = {k: v / total_auc for k, v in val_aucs.items()} # Ensure weights sum to 1
weighted_probs = np.zeros_like(avg_probs)
for name, prob in test_probs_dict.items():
    if name in weights: # Ensure we only use models that contributed to weights
        weighted_probs += prob * weights[name]
test_probs_dict['Weighted Ensemble'] = weighted_probs # Store test probs


# Prepare Data for Stacking Meta-Learners
X_meta_train_stack = np.column_stack(list(val_probs_dict.values())) # Base model predictions on validation set
y_meta_train_stack = y_val
X_meta_test_stack = np.column_stack(list(test_probs_dict.values())[:-2]) # Base model predictions on test set (excluding current ensembles)

# C. Stacked Ensemble (Logistic Regression Meta) - Train & Calc Test Probs
meta_lr = LogisticRegression(random_state=42)
meta_lr.fit(X_meta_train_stack, y_meta_train_stack)
stack_lr_probs = meta_lr.predict_proba(X_meta_test_stack)[:, 1]
test_probs_dict['Stacked (LR)'] = stack_lr_probs # Store test probs

# D. Stacked Ensemble (XGBoost Meta) - Train & Calc Test Probs
meta_xgb = XGBClassifier(eval_metric='logloss', random_state=42)
meta_xgb.fit(X_meta_train_stack, y_meta_train_stack)
stack_xgb_probs = meta_xgb.predict_proba(X_meta_test_stack)[:, 1]
test_probs_dict['Stacked (XGB)'] = stack_xgb_probs # Store test probs

# --- E. TUNE ENSEMBLE THRESHOLDS (MAX F1 on Validation Set) ---
print("\n--- Tuning Ensemble Thresholds for F1-score (on Validation Set) ---")

# Calculate validation probabilities for ensembles for tuning
val_avg_probs = np.mean(list(val_probs_dict.values()), axis=0)
val_weighted_probs = np.zeros_like(val_avg_probs)
for name, prob in val_probs_dict.items(): # Use val_probs_dict for averaging
    if name in weights: # Ensure weights are applied correctly
        val_weighted_probs += prob * weights[name]

# Stacked meta-learners already trained on X_meta_train_stack (validation predictions)
val_stack_lr_probs = meta_lr.predict_proba(X_meta_train_stack)[:, 1]
val_stack_xgb_probs = meta_xgb.predict_proba(X_meta_train_stack)[:, 1]

ensemble_val_probs_for_tuning = {
    'Averaged Ensemble': val_avg_probs,
    'Weighted Ensemble': val_weighted_probs,
    'Stacked (LR)': val_stack_lr_probs,
    'Stacked (XGB)': val_stack_xgb_probs
}

for name in ensemble_names_list:
    probs = ensemble_val_probs_for_tuning[name]
    precision, recall, thresholds = precision_recall_curve(y_val, probs)
    # Handle division by zero for fscore calculation, especially for cases with no positive predictions
    fscore = np.divide(2 * precision * recall, precision + recall, out=np.zeros_like(precision), where=(precision + recall != 0))

    # Check if fscore array is empty or contains only NaNs (e.g., if no positive predictions or recall=0)
    if fscore.size > 0 and not np.all(np.isnan(fscore)):
        best_thresh_idx = np.nanargmax(fscore) # Use nanargmax to ignore NaNs
        # Ensure best_thresh_idx is within thresholds array bounds
        if best_thresh_idx < len(thresholds):
            best_thresholds[name] = thresholds[best_thresh_idx]
        else:
            best_thresholds[name] = 0.5 # Default if index is out of bounds
    else:
        best_thresholds[name] = 0.5 # Default threshold if no meaningful F1 can be calculated
    print(f"  {name:<20}: Optimal F1 Threshold = {best_thresholds[name]:.4f}")


# ==========================================================
# --- FINAL RESULTS TABLE (Individual Models + Ensembles) ---
# ==========================================================
print("\n--- INDIVIDUAL & ENSEMBLE MODEL RESULTS ---")
print(f"{'Model':<20} | {'ROC AUC':<10} | {'PR AUC':<10} | {'F1-Score':<10} | {'Accuracy':<10} | {'W-F1':<10} | {'M-F1':<10}") # Updated header
print("-" * 98)

# Individual Models
for name in best_models.keys():
    if name == "One-Class SVM":
        # OCSVM Logic
        scores = -best_models[name].decision_function(X_test_feat)
        test_preds = np.where(best_models[name].predict(X_test_feat) == -1, 1, 0)
        roc = roc_auc_score(y_test, scores)
        pr = average_precision_score(y_test, scores)
        f1 = f1_score(y_test, test_preds)
        acc = accuracy_score(y_test, test_preds)
        f1_w = f1_score(y_test, test_preds, average='weighted')
        f1_m = f1_score(y_test, test_preds, average='macro')
    else:
        probs = test_probs_dict[name]
        # Apply the tuned threshold to get binary predictions for F1, Accuracy
        binary_preds = (probs >= best_thresholds[name]).astype(int)

        roc = roc_auc_score(y_test, probs)
        pr = average_precision_score(y_test, probs)
        f1 = f1_score(y_test, binary_preds)
        acc = accuracy_score(y_test, binary_preds)
        f1_w = f1_score(y_test, binary_preds, average='weighted')
        f1_m = f1_score(y_test, binary_preds, average='macro')

    print(f"{name:<20} | {roc:.4f}     | {pr:.4f}     | {f1:.4f}     | {acc:.4f}     | {f1_w:.4f}     | {f1_m:.4f}")

# Ensembles
for name in ensemble_names_list:
    probs = test_probs_dict[name]
    # Apply the tuned F1 threshold for ensembles to get binary predictions
    binary_preds = (probs >= best_thresholds[name]).astype(int)

    roc = roc_auc_score(y_test, probs)
    pr = average_precision_score(y_test, probs)
    f1 = f1_score(y_test, binary_preds)
    acc = accuracy_score(y_test, binary_preds)
    f1_w = f1_score(y_test, binary_preds, average='weighted')
    f1_m = f1_score(y_test, binary_preds, average='macro')

    print(f"{name:<20} | {roc:.4f}     | {pr:.4f}     | {f1:.4f}     | {acc:.4f}     | {f1_w:.4f}     | {f1_m:.4f}")

print("-" * 98)

In [ ]:
# @title 9. Generate Final ROC & PR Curves (All Models + Advanced Ensembles)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score

# Setup Plot (1 Row, 2 Columns)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 9)) # Slightly larger figure for better spacing
plt.rcParams.update({'font.size': 14, 'font.family': 'serif'}) # Increased overall font size

# --- DEFINE COLORS & STYLES ---
colors = {
    'Logistic Regression': '#1f77b4',  # Blue
    'Random Forest':       '#ff7f0e',  # Orange
    'XGBoost':             '#2ca02c',  # Green
    'MLP':                 '#d62728',  # Red
    'KNN':                 '#9467bd',  # Purple
    'One-Class SVM':       '#e377c2',  # Pink
    'Averaged Ensemble':   'black',
    'Weighted Ensemble':   '#8c564b',  # Brown
    'Stacked (LR)':        '#7f7f7f',  # Gray
    'Stacked (XGB)':       '#bcbd22'   # Olive
}

# Styles for differentiating Ensembles
styles = {
    'Averaged Ensemble': '-',
    'Weighted Ensemble': '--',
    'Stacked (LR)':      '-.',
    'Stacked (XGB)':     ':'
}

# ==========================================
# PLOT 1: ROC CURVES (Left Panel)
# ==========================================

# 1. Baseline (LSTM-AE)
fpr_ae, tpr_ae, _ = roc_curve(y_test, test_mae)
auc_ae = roc_auc_score(y_test, test_mae)
ax1.plot(fpr_ae, tpr_ae, label=f'LSTM-AE Baseline (AUC = {auc_ae:.3f})',
         linestyle='--', color='gray', linewidth=2.5, alpha=0.7) # Increased linewidth

# 2. Individual Tuned Classifiers
for name, clf in best_models.items():
    if name == "One-Class SVM":
        # OCSVM Logic
        scores = -clf.decision_function(X_test_feat)
        fpr, tpr, _ = roc_curve(y_test, scores)
        roc_val = roc_auc_score(y_test, scores)
        ax1.plot(fpr, tpr, label=f'{name} (AUC = {roc_val:.3f})',
                 color=colors.get(name, 'black'), linewidth=1.5, linestyle=':', alpha=0.6) # Increased linewidth
        continue

    # Standard Models
    probs = clf.predict_proba(X_test_feat)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_val = roc_auc_score(y_test, probs)

    # Highlight RF/XGB slightly
    lw = 2.5 if name in ['Random Forest', 'XGBoost'] else 1.5 # Increased linewidth
    ax1.plot(fpr, tpr, label=f'{name} (AUC = {roc_val:.3f})',
             color=colors.get(name, 'black'), linewidth=lw, alpha=0.8)

# 3. Advanced Ensembles (From test_probs_dict)
ensemble_names = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']

for name in ensemble_names:
    if name in test_probs_dict:
        probs = test_probs_dict[name]
        fpr, tpr, _ = roc_curve(y_test, probs)
        roc_val = roc_auc_score(y_test, probs)

        # Make Averaged Ensemble thickest
        lw = 4 if name == 'Averaged Ensemble' else 3 # Increased linewidth
        ax1.plot(fpr, tpr, label=f'{name} (AUC = {roc_val:.3f})',
                 color=colors.get(name, 'black'), linestyle=styles.get(name, '-'),
                 linewidth=lw, zorder=10)

# Formatting ROC
ax1.plot([0, 1], [0, 1], 'k:', alpha=0.4)
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.02])
ax1.set_xlabel('False Positive Rate (1 - Specificity)', fontweight='bold')
ax1.set_ylabel('True Positive Rate (Sensitivity)', fontweight='bold')
ax1.set_title('Receiver Operating Characteristic (ROC)', fontweight='bold', pad=10)
ax1.legend(loc="lower right", fontsize=11, ncol=1) # Increased legend fontsize
ax1.grid(True, alpha=0.3)


# ==========================================
# PLOT 2: PRECISION-RECALL CURVES (Right Panel)
# ==========================================

# 1. Baseline
precision_ae, recall_ae, _ = precision_recall_curve(y_test, test_mae)
pr_auc_ae = average_precision_score(y_test, test_mae)
ax2.plot(recall_ae, precision_ae, label=f'LSTM-AE Baseline (AP = {pr_auc_ae:.3f})',
         linestyle='--', color='gray', linewidth=2.5, alpha=0.7) # Increased linewidth

# 2. Individual Classifiers
for name, clf in best_models.items():
    if name == "One-Class SVM":
        scores = -clf.decision_function(X_test_feat)
        precision, recall, _ = precision_recall_curve(y_test, scores)
        pr_val = average_precision_score(y_test, scores)
        ax2.plot(recall, precision, label=f'{name} (AP = {pr_val:.3f})',
                 color=colors.get(name, 'black'), linewidth=1.5, linestyle=':', alpha=0.6) # Increased linewidth
        continue

    probs = clf.predict_proba(X_test_feat)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, probs)
    pr_val = average_precision_score(y_test, probs)

    lw = 2.5 if name in ['Random Forest', 'XGBoost'] else 1.5 # Increased linewidth
    ax2.plot(recall, precision, label=f'{name} (AP = {pr_val:.3f})',
             color=colors.get(name, 'black'), linewidth=lw, alpha=0.8)

# 3. Advanced Ensembles
for name in ensemble_names:
    if name in test_probs_dict:
        probs = test_probs_dict[name]
        precision, recall, _ = precision_recall_curve(y_test, probs)
        pr_val = average_precision_score(y_test, probs)

        lw = 4 if name == 'Averaged Ensemble' else 3 # Increased linewidth
        ax2.plot(recall, precision, label=f'{name} (AP = {pr_val:.3f})',
                 color=colors.get(name, 'black'), linestyle=styles.get(name, '-'),
                 linewidth=lw, zorder=10)

# Formatting PR
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.02])
ax2.set_xlabel('Recall (Sensitivity)', fontweight='bold')
ax2.set_ylabel('Precision (Positive Predictive Value)', fontweight='bold')
ax2.set_title('Precision-Recall (PR) Curves', fontweight='bold', pad=10)
ax2.legend(loc="lower left", fontsize=11, ncol=1) # Increased legend fontsize
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Figure3b_Complete_Ensemble_Analysis.pdf', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, classification_report

# List of ensembles to tune
ensemble_keys = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']

print("--- TUNING ENSEMBLE THRESHOLDS (Target: Max F1-Score) ---")
print("-" * 60)

for name in ensemble_keys:
    if name in test_probs_dict:
        # 1. Extract Probabilities
        probs = test_probs_dict[name]

        # Ensure 1D array
        if isinstance(probs, np.ndarray) and probs.ndim == 2:
            probs = probs[:, 1]

        # 2. Compute Precision-Recall Curve
        precision, recall, thresholds = precision_recall_curve(y_test, probs)

        # 3. Calculate F1 Score for every possible threshold
        # We ignore the last value of precision/recall as it has no corresponding threshold
        numerator = 2 * precision[:-1] * recall[:-1]
        denominator = precision[:-1] + recall[:-1]

        # Handle division by zero
        fscore = np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)

        # 4. Find the Index of the Maximum F1 Score
        ix = np.argmax(fscore)
        best_thresh = thresholds[ix]
        best_f1 = fscore[ix]

        # 5. Update your global dictionary
        best_thresholds[name] = best_thresh

        print(f"{name:20} | New Threshold: {best_thresh:.4f} | Max F1: {best_f1:.4f}")

print("=" * 60)
print("Ensemble thresholds updated in 'best_thresholds'.")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

# ==========================================
# 1. SETUP & MERGE PREDICTIONS
# ==========================================
# Combine trained models and ensemble probabilities into one dictionary
all_eval_items = {}
all_eval_items.update(best_models) # Your supervised models (RF, SVM, etc.)

# Add specific ensembles from your probability dictionary
# Ensure keys match those in 'best_thresholds' if you tuned them
ensemble_keys = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']
for key in ensemble_keys:
    if key in test_probs_dict:
        all_eval_items[key] = test_probs_dict[key]

print("--- DETAILED CLASSIFICATION REPORTS ---")
print("Note: Thresholds are retrieved from 'best_thresholds'. Defaults to 0.5 if not found.")
print(f"{'='*60}")

# ==========================================
# 2. EVALUATION LOOP
# ==========================================
for name, artifact in all_eval_items.items():

    print(f"\nModel: {name}")
    print("-" * 30)

    # --------------------------------------
    # CASE A: One-Class SVM (Anomaly Detection)
    # --------------------------------------
    if name == "One-Class SVM":
        # OCSVM returns -1 (outlier/dysbiosis) and 1 (inlier/normal)
        # We map -1 to 1 (Positive Class) and 1 to 0 (Negative Class)
        raw_preds = artifact.predict(X_test_feat)
        binary_preds = np.where(raw_preds == -1, 1, 0)

    # --------------------------------------
    # CASE B: Pre-calculated Probabilities (Ensembles)
    # --------------------------------------
    elif isinstance(artifact, np.ndarray):
        # These are probabilities (N,) or (N, 2)
        # If shape is (N, 2), take the second column
        if artifact.ndim == 2 and artifact.shape[1] == 2:
            probs = artifact[:, 1]
        else:
            probs = artifact

        # Apply the specific threshold tuned for this ensemble
        # If you haven't tuned the ensemble threshold, this defaults to 0.5
        thresh = best_thresholds.get(name, 0.5)
        binary_preds = (probs >= thresh).astype(int)

        print(f"Type: Ensemble Probabilities | Threshold: {thresh:.4f}")

    # --------------------------------------
    # CASE C: Standard Supervised Models
    # --------------------------------------
    else:
        # Check if model supports probability prediction
        if hasattr(artifact, "predict_proba"):
            probs = artifact.predict_proba(X_test_feat)[:, 1]

            # Apply the specific threshold tuned for this model
            thresh = best_thresholds.get(name, 0.5)
            binary_preds = (probs >= thresh).astype(int)

            print(f"Type: Sklearn Model | Threshold: {thresh:.4f}")

        # Fallback for models like LinearSVC without probability=True
        elif hasattr(artifact, "predict"):
            binary_preds = artifact.predict(X_test_feat)
            print("Type: Hard Prediction (No Probability Support)")

        else:
            print(f"Error: Unknown artifact type for {name}")
            continue

    # --------------------------------------
    # 3. GENERATE REPORT
    # --------------------------------------
    report = classification_report(
        y_test,
        binary_preds,
        target_names=['Class 0 (Normal)', 'Class 1 (Dysbiotic)'],
        zero_division=0
    )
    print(report)
    print("="*60)

In [ ]:
print("\n--- RE-EVALUATING ENSEMBLES WITH OPTIMAL THRESHOLDS ---")

for name in ensemble_keys:
    if name in test_probs_dict:
        print(f"\nModel: {name}")
        print(f"Threshold Used: {best_thresholds[name]:.4f}")
        print("-" * 30)

        # Get probs
        probs = test_probs_dict[name]
        if probs.ndim == 2: probs = probs[:, 1]

        # Apply NEW threshold
        binary_preds = (probs >= best_thresholds[name]).astype(int)

        # Report
        print(classification_report(
            y_test,
            binary_preds,
            target_names=['Class 0 (Normal)', 'Class 1 (Dysbiotic)'],
            zero_division=0
        ))
        print("="*60)

In [ ]:
# @title 15. Feature Importance Analysis (Reconstruction Attribution)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. CALCULATE ERROR PER FEATURE ---
# Reconstruct the Test Set
reconstructions = autoencoder.predict(X_test, verbose=0)

# Calculate MAE for each feature (averaged over time only)
# Result Shape: (n_samples, n_features)
mae_per_feature = np.mean(np.abs(X_test - reconstructions), axis=1)

# --- 2. IDENTIFY DYSBIOTIC SAMPLES ---
# We focus on samples that were TRULY Dysbiotic (y_test == 1)
# to see what characterizes the disease state.
dysbiosis_indices = np.where(y_test == 1)[0]
dysbiosis_errors = mae_per_feature[dysbiosis_indices]

# Calculate Mean Error per feature across all Dysbiotic samples
mean_error_per_feature = np.mean(dysbiosis_errors, axis=0)

# --- 3. MAP TO NAMES ---
# Create a DataFrame
# Note: 'feature_cols' comes from your Step 3 (Patient Split) block
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Mean_MAE': mean_error_per_feature
})

# Sort by Error (High Error = High Importance)
importance_df = importance_df.sort_values(by='Mean_MAE', ascending=False).head(20)

# --- 4. PLOT ---
plt.figure(figsize=(10, 8))
plt.rcParams.update({'font.size': 12, 'font.family': 'serif'})

sns.barplot(data=importance_df, x='Mean_MAE', y='Feature', palette='viridis')

plt.title('Top 20 Bacteria Driving Dysbiosis Classification\n(Based on Reconstruction Error Contribution)', fontweight='bold')
plt.xlabel('Mean Reconstruction Error (MAE)', fontweight='bold')
plt.ylabel('Bacterial Genus', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Figure4_Feature_Importance.pdf', dpi=600)
plt.show()

print("Top 5 Drivers:")
print(importance_df[['Feature', 'Mean_MAE']].head(5))

In [ ]:
# @title 10. Statistical Bootstrap Analysis (All Ensembles & Models)
import numpy as np
from scipy.stats import wilcoxon
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm import tqdm

# --- CONFIGURATION ---
n_iterations = 1000
alpha = 0.05

# Define all models to test
ensemble_names = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']
# Combine Baseline + Individual Models + Ensembles
model_names = ['LSTM-AE Baseline'] + list(best_models.keys()) + ensemble_names

# Store metric distributions
bootstrap_results = {
    'ROC AUC': {name: [] for name in model_names},
    'PR AUC': {name: [] for name in model_names}
}

print(f"Starting Bootstrap Analysis ({n_iterations} iterations)...")

# --- BOOTSTRAP LOOP ---
n_test = len(y_test)
np.random.seed(42)

# Identify which models contribute to ensembles (All except OCSVM)
supervised_model_names = [name for name in best_models.keys() if name != "One-Class SVM"]

for i in tqdm(range(n_iterations)):
    # 1. Resample indices
    indices = np.random.choice(n_test, n_test, replace=True)
    y_true_boot = y_test[indices]
    X_feat_boot = X_test_feat[indices]

    # Edge case check
    if len(np.unique(y_true_boot)) < 2: continue

    # --- A. BASELINE ---
    baseline_scores = test_mae[indices]
    bootstrap_results['ROC AUC']['LSTM-AE Baseline'].append(roc_auc_score(y_true_boot, baseline_scores))
    bootstrap_results['PR AUC']['LSTM-AE Baseline'].append(average_precision_score(y_true_boot, baseline_scores))

    # --- B. INDIVIDUAL CLASSIFIERS ---
    boot_probs_list = [] # List to store probabilities for Ensembling

    for name, clf in best_models.items():
        if name == "One-Class SVM":
            # OCSVM Logic (Not part of Ensembles)
            scores = -clf.decision_function(X_feat_boot)
            bootstrap_results['ROC AUC'][name].append(roc_auc_score(y_true_boot, scores))
            bootstrap_results['PR AUC'][name].append(average_precision_score(y_true_boot, scores))
        else:
            # Supervised Models
            scores = clf.predict_proba(X_feat_boot)[:, 1]
            boot_probs_list.append(scores)

            # Store Individual Result
            bootstrap_results['ROC AUC'][name].append(roc_auc_score(y_true_boot, scores))
            bootstrap_results['PR AUC'][name].append(average_precision_score(y_true_boot, scores))

    # --- C. ENSEMBLES ---

    # 1. Averaged Ensemble
    avg_scores = np.mean(boot_probs_list, axis=0)
    bootstrap_results['ROC AUC']['Averaged Ensemble'].append(roc_auc_score(y_true_boot, avg_scores))
    bootstrap_results['PR AUC']['Averaged Ensemble'].append(average_precision_score(y_true_boot, avg_scores))

    # 2. Weighted Ensemble
    # Uses 'weights' dictionary from Step 7
    w_scores = np.zeros_like(avg_scores)
    # Reconstruct weighted sum using the known order of supervised_model_names
    for idx, name in enumerate(supervised_model_names):
        w_scores += boot_probs_list[idx] * weights[name]

    bootstrap_results['ROC AUC']['Weighted Ensemble'].append(roc_auc_score(y_true_boot, w_scores))
    bootstrap_results['PR AUC']['Weighted Ensemble'].append(average_precision_score(y_true_boot, w_scores))

    # Prepare Input for Stacking Meta-Learners
    X_meta_boot = np.column_stack(boot_probs_list)

    # 3. Stacked (LR)
    slr_scores = meta_lr.predict_proba(X_meta_boot)[:, 1]
    bootstrap_results['ROC AUC']['Stacked (LR)'].append(roc_auc_score(y_true_boot, slr_scores))
    bootstrap_results['PR AUC']['Stacked (LR)'].append(average_precision_score(y_true_boot, slr_scores))

    # 4. Stacked (XGB)
    sxgb_scores = meta_xgb.predict_proba(X_meta_boot)[:, 1]
    bootstrap_results['ROC AUC']['Stacked (XGB)'].append(roc_auc_score(y_true_boot, sxgb_scores))
    bootstrap_results['PR AUC']['Stacked (XGB)'].append(average_precision_score(y_true_boot, sxgb_scores))


# --- STATISTICAL SIGNIFICANCE ---
# We compare everyone against the "Averaged Ensemble" (or whichever won in Step 7)
print("\nCalculating P-Values (Comparison vs. Averaged Ensemble)...")

p_values = {}
ref_roc = bootstrap_results['ROC AUC']['Averaged Ensemble']
ref_pr = bootstrap_results['PR AUC']['Averaged Ensemble']

for name in model_names:
    if name == 'Averaged Ensemble': continue

    # Paired Wilcoxon Test
    _, p_roc = wilcoxon(bootstrap_results['ROC AUC'][name], ref_roc)
    _, p_pr = wilcoxon(bootstrap_results['PR AUC'][name], ref_pr)

    p_values[name] = {'p_roc': p_roc, 'p_pr': p_pr}

# --- FINAL REPORT GENERATION ---
print("\n" + "="*95)
print(f"{'Model':<25} | {'ROC AUC (95% CI)':<28} | {'PR AUC (95% CI)':<28}")
print("-" * 95)

def get_ci(data):
    lower = np.percentile(data, 2.5)
    mean = np.mean(data)
    upper = np.percentile(data, 97.5)
    return f"{mean:.3f} [{lower:.3f}, {upper:.3f}]"

# Print Table
for name in model_names:
    row_roc = get_ci(bootstrap_results['ROC AUC'][name])
    row_pr = get_ci(bootstrap_results['PR AUC'][name])

    if name == 'Averaged Ensemble':
        print(f"{name:<25} | {row_roc:<28} | {row_pr:<28}")
    else:
        # Mark significance
        sig = "*" if p_values[name]['p_roc'] < 0.05 else ""
        print(f"{name:<25} | {row_roc:<28}{sig} | {row_pr:<28}")

print("="*95)
print("* p < 0.05 compared to Averaged Ensemble (Wilcoxon Signed-Rank Test)")

In [ ]:
import numpy as np
from scipy.stats import wilcoxon, ttest_rel
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_recall_curve # Added f1_score and precision_recall_curve
from tqdm import tqdm

# --- CONFIGURATION ---
n_iterations = 1000
alpha = 0.05

# Define all models to test
# Base models + Ensembles
ensemble_names = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']
model_names = ['LSTM-AE Baseline'] + list(best_models.keys()) + ensemble_names

# Store metric distributions
bootstrap_results = {
    'ROC AUC': {name: [] for name in model_names},
    'PR AUC': {name: [] for name in model_names},
    'F1-Score': {name: [] for name in model_names} # Added F1-Score
}

print(f"Starting Bootstrap Analysis ({n_iterations} iterations)...")

# --- TUNE F1 THRESHOLDS ON VALIDATION SET FOR ALL MODELS (IF NOT ALREADY TUNED) ---
# This is crucial for consistent F1-score calculation during bootstrap
print("\n--- Tuning F1 Thresholds on Validation Set for all models ---")

# Tune F1 Threshold for LSTM-AE Baseline
precision_ae_val, recall_ae_val, thresholds_ae_val = precision_recall_curve(y_val, val_mae)
fscore_ae_val = np.divide(2 * precision_ae_val * recall_ae_val, precision_ae_val + recall_ae_val, out=np.zeros_like(precision_ae_val), where=(precision_ae_val + recall_ae_val != 0))
if fscore_ae_val.size > 0 and not np.all(np.isnan(fscore_ae_val)):
    best_thresh_idx_ae_val = np.nanargmax(fscore_ae_val) # Use nanargmax to ignore NaNs
    if best_thresh_idx_ae_val < len(thresholds_ae_val):
        best_thresholds['LSTM-AE Baseline'] = thresholds_ae_val[best_thresh_idx_ae_val]
    else:
        best_thresholds['LSTM-AE Baseline'] = 0.5 # Default if index is out of bounds
else:
    best_thresholds['LSTM-AE Baseline'] = 0.5 # Default threshold if no meaningful F1 can be calculated
print(f"  {'LSTM-AE Baseline':<20}: Optimal F1 Threshold = {best_thresholds['LSTM-AE Baseline']:.4f}")

# Tune F1 Thresholds for Individual Supervised Models (if not already set for F1)
supervised_models_for_f1_tuning = [name for name in best_models.keys() if name != "One-Class SVM"]

for name in supervised_models_for_f1_tuning:
    # Assuming val_probs_dict contains probabilities for individual models from TG4lfZc7R8g6
    if name in val_probs_dict:
        probs = val_probs_dict[name]
        precision, recall, thresholds = precision_recall_curve(y_val, probs)
        fscore = np.divide(2 * precision * recall, precision + recall, out=np.zeros_like(precision), where=(precision + recall != 0))

        if fscore.size > 0 and not np.all(np.isnan(fscore)):
            best_thresh_idx = np.nanargmax(fscore)
            if best_thresh_idx < len(thresholds):
                best_thresholds[name] = thresholds[best_thresh_idx]
            else:
                best_thresholds[name] = 0.5
        else:
            best_thresholds[name] = 0.5
        print(f"  {name:<20}: Optimal F1 Threshold = {best_thresholds[name]:.4f}")


# --- BOOTSTRAP LOOP ---
n_test = len(y_test)
np.random.seed(42)

# Identify supervised models for stacking (must match order in Step 7)
supervised_models = [name for name in best_models.keys() if name != "One-Class SVM"]

for i in tqdm(range(n_iterations)):
    # 1. Resample
    indices = np.random.choice(n_test, n_test, replace=True)
    y_true_boot = y_test[indices]
    X_feat_boot = X_test_feat[indices]

    # Check bounds (edge case: only one class in sample)
    if len(np.unique(y_true_boot)) < 2: continue

    # --- A. BASELINE (LSTM-AE) ---
    baseline_scores = test_mae[indices]
    bootstrap_results['ROC AUC']['LSTM-AE Baseline'].append(roc_auc_score(y_true_boot, baseline_scores))
    bootstrap_results['PR AUC']['LSTM-AE Baseline'].append(average_precision_score(y_true_boot, baseline_scores))

    # F1-Score for baseline
    baseline_binary_preds = (baseline_scores >= best_thresholds['LSTM-AE Baseline']).astype(int)
    bootstrap_results['F1-Score']['LSTM-AE Baseline'].append(f1_score(y_true_boot, baseline_binary_preds))

    # --- B. INDIVIDUAL CLASSIFIERS ---
    boot_probs_list = [] # Store probs for ensembles (order matters!)

    for name, clf in best_models.items():
        if name == "One-Class SVM":
            # OCSVM Logic
            scores_ocsvm = -clf.decision_function(X_feat_boot)
            binary_preds_ocsvm = np.where(clf.predict(X_feat_boot) == -1, 1, 0)
            bootstrap_results['ROC AUC'][name].append(roc_auc_score(y_true_boot, scores_ocsvm))
            bootstrap_results['PR AUC'][name].append(average_precision_score(y_true_boot, scores_ocsvm))
            bootstrap_results['F1-Score'][name].append(f1_score(y_true_boot, binary_preds_ocsvm))
        else:
            # Standard Supervised Models
            scores_supervised = clf.predict_proba(X_feat_boot)[:, 1]
            boot_probs_list.append(scores_supervised) # Append for ensemble stacking

            binary_preds_supervised = (scores_supervised >= best_thresholds[name]).astype(int)

            bootstrap_results['ROC AUC'][name].append(roc_auc_score(y_true_boot, scores_supervised))
            bootstrap_results['PR AUC'][name].append(average_precision_score(y_true_boot, scores_supervised))
            bootstrap_results['F1-Score'][name].append(f1_score(y_true_boot, binary_preds_supervised))

    # --- C. ENSEMBLES ---

    # 1. Averaged Ensemble
    avg_scores = np.mean(boot_probs_list, axis=0)
    binary_preds_avg = (avg_scores >= best_thresholds['Averaged Ensemble']).astype(int)
    bootstrap_results['ROC AUC']['Averaged Ensemble'].append(roc_auc_score(y_true_boot, avg_scores))
    bootstrap_results['PR AUC']['Averaged Ensemble'].append(average_precision_score(y_true_boot, avg_scores))
    bootstrap_results['F1-Score']['Averaged Ensemble'].append(f1_score(y_true_boot, binary_preds_avg))

    # 2. Weighted Ensemble
    # Uses 'weights' dictionary calculated in Step 7
    w_scores = np.zeros_like(avg_scores)
    for idx, name in enumerate(supervised_models):
        w_scores += boot_probs_list[idx] * weights[name]
    binary_preds_w = (w_scores >= best_thresholds['Weighted Ensemble']).astype(int)
    bootstrap_results['ROC AUC']['Weighted Ensemble'].append(roc_auc_score(y_true_boot, w_scores))
    bootstrap_results['PR AUC']['Weighted Ensemble'].append(average_precision_score(y_true_boot, w_scores))
    bootstrap_results['F1-Score']['Weighted Ensemble'].append(f1_score(y_true_boot, binary_preds_w))

    # Prepare Input for Meta-Learners (Stacking)
    X_meta_boot = np.column_stack(boot_probs_list)

    # 3. Stacked (LR)
    slr_scores = meta_lr.predict_proba(X_meta_boot)[:, 1]
    binary_preds_slr = (slr_scores >= best_thresholds['Stacked (LR)']).astype(int)
    bootstrap_results['ROC AUC']['Stacked (LR)'].append(roc_auc_score(y_true_boot, slr_scores))
    bootstrap_results['PR AUC']['Stacked (LR)'].append(average_precision_score(y_true_boot, slr_scores))
    bootstrap_results['F1-Score']['Stacked (LR)'].append(f1_score(y_true_boot, binary_preds_slr))

    # 4. Stacked (XGB)
    sxgb_scores = meta_xgb.predict_proba(X_meta_boot)[:, 1]
    binary_preds_sxgb = (sxgb_scores >= best_thresholds['Stacked (XGB)']).astype(int)
    bootstrap_results['ROC AUC']['Stacked (XGB)'].append(roc_auc_score(y_true_boot, sxgb_scores))
    bootstrap_results['PR AUC']['Stacked (XGB)'].append(average_precision_score(y_true_boot, sxgb_scores))
    bootstrap_results['F1-Score']['Stacked (XGB)'].append(f1_score(y_true_boot, binary_preds_sxgb))


# --- STATISTICAL COMPARISON ---
print("\n" + "="*115)
print(f"{'Model':<25} | {'Metric':<10} | {'Value (95% CI)':<25} | {'p-value (Wilcoxon)':<20} | {'p-value (t-test)':<20}")
print("-" * 115)

# Reference Model: Averaged Ensemble (You can change this to Weighted or Stacked if they perform better)
ref_roc = bootstrap_results['ROC AUC']['Averaged Ensemble']
ref_pr = bootstrap_results['PR AUC']['Averaged Ensemble']
ref_f1 = bootstrap_results['F1-Score']['Averaged Ensemble']

def get_ci(data):
    lower = np.percentile(data, 2.5)
    mean = np.mean(data)
    upper = np.percentile(data, 97.5)
    return f"{mean:.3f} [{lower:.3f}, {upper:.3f}]"

metrics_to_report = ['ROC AUC', 'PR AUC', 'F1-Score']

# Print Table
for name in model_names:
    first_metric_printed = False
    for metric_name in metrics_to_report:
        metric_data = bootstrap_results[metric_name][name]
        ci_str = get_ci(metric_data)

        p_wilcox = 1.0
        p_ttest = 1.0
        sig_mark = ""

        if name == 'Averaged Ensemble':
            # Reference model - p-values are 1.0 by definition
            pass
        else:
            # Paired Tests against Reference for the specific metric
            ref_metric_data = None
            if metric_name == 'ROC AUC': ref_metric_data = ref_roc
            elif metric_name == 'PR AUC': ref_metric_data = ref_pr
            elif metric_name == 'F1-Score': ref_metric_data = ref_f1

            _, p_wilcox = wilcoxon(metric_data, ref_metric_data)
            _, p_ttest = ttest_rel(metric_data, ref_metric_data)
            sig_mark = "*" if p_wilcox < 0.05 else ""

        model_col = name if not first_metric_printed else ""
        first_metric_printed = True

        print(f"{model_col:<25} | {metric_name:<10} | {ci_str:<25} | {p_wilcox:.2e} {sig_mark:<10} | {p_ttest:.2e}")

print("="*115)
print("* p < 0.05 compared to Averaged Ensemble (Significant Difference)")

In [ ]:
import numpy as np
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score # Added f1_score
from tqdm import tqdm

# --- CONFIGURATION ---
n_iterations = 1000
alpha = 0.05 # Significance level

# Define all models to test
ensemble_names = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)'] # Include all ensembles here
model_names = ['LSTM-AE Baseline'] + list(best_models.keys()) + ensemble_names # Ensure all models are included

# Store metric distributions
bootstrap_results = {
    'ROC AUC': {name: [] for name in model_names},
    'PR AUC': {name: [] for name in model_names},
    'F1-Score': {name: [] for name in model_names} # Added F1-Score
}

print(f"Starting Bootstrap Analysis ({n_iterations} iterations)...")

# --- BOOTSTRAP LOOP ---
n_test = len(y_test)
np.random.seed(42)

# Identify supervised models for stacking (must match order in Step 7)
supervised_models_for_ensembling = [name for name in best_models.keys() if name != "One-Class SVM"]

for i in tqdm(range(n_iterations)):
    # 1. Resample
    indices = np.random.choice(n_test, n_test, replace=True)
    y_true_boot = y_test[indices]
    X_feat_boot = X_test_feat[indices]

    # Check bounds (edge case: only one class in sample)
    if len(np.unique(y_true_boot)) < 2: continue

    # --- A. BASELINE (LSTM-AE) ---
    baseline_scores = test_mae[indices]
    bootstrap_results['ROC AUC']['LSTM-AE Baseline'].append(roc_auc_score(y_true_boot, baseline_scores))
    bootstrap_results['PR AUC']['LSTM-AE Baseline'].append(average_precision_score(y_true_boot, baseline_scores))
    # F1-Score for baseline (using pre-tuned threshold)
    baseline_binary_preds = (baseline_scores >= best_thresholds['LSTM-AE Baseline']).astype(int)
    bootstrap_results['F1-Score']['LSTM-AE Baseline'].append(f1_score(y_true_boot, baseline_binary_preds))

    # --- B. INDIVIDUAL CLASSIFIERS ---
    boot_probs_list = [] # Store probs for supervised models for ensembling

    for name, clf in best_models.items():
        if name == "One-Class SVM":
            # OCSVM Logic
            scores_ocsvm = -clf.decision_function(X_feat_boot)
            binary_preds_ocsvm = np.where(clf.predict(X_feat_boot) == -1, 1, 0)
            bootstrap_results['ROC AUC'][name].append(roc_auc_score(y_true_boot, scores_ocsvm))
            bootstrap_results['PR AUC'][name].append(average_precision_score(y_true_boot, scores_ocsvm))
            bootstrap_results['F1-Score'][name].append(f1_score(y_true_boot, binary_preds_ocsvm))
        else:
            # Standard Supervised Models
            scores_supervised = clf.predict_proba(X_feat_boot)[:, 1]
            boot_probs_list.append(scores_supervised) # Append for ensemble stacking

            # F1-Score for supervised models (using pre-tuned threshold)
            binary_preds_supervised = (scores_supervised >= best_thresholds[name]).astype(int)

            bootstrap_results['ROC AUC'][name].append(roc_auc_score(y_true_boot, scores_supervised))
            bootstrap_results['PR AUC'][name].append(average_precision_score(y_true_boot, scores_supervised))
            bootstrap_results['F1-Score'][name].append(f1_score(y_true_boot, binary_preds_supervised))

    # --- C. ENSEMBLES ---

    # 1. Averaged Ensemble
    avg_scores = np.mean(boot_probs_list, axis=0)
    binary_preds_avg = (avg_scores >= best_thresholds['Averaged Ensemble']).astype(int)
    bootstrap_results['ROC AUC']['Averaged Ensemble'].append(roc_auc_score(y_true_boot, avg_scores))
    bootstrap_results['PR AUC']['Averaged Ensemble'].append(average_precision_score(y_true_boot, avg_scores))
    bootstrap_results['F1-Score']['Averaged Ensemble'].append(f1_score(y_true_boot, binary_preds_avg))

    # 2. Weighted Ensemble
    # Uses 'weights' dictionary from Step 7 (assuming it's available and ordered correctly)
    w_scores = np.zeros_like(avg_scores)
    for idx, name in enumerate(supervised_models_for_ensembling):
        if name in weights: # Ensure model contributed to weights
             w_scores += boot_probs_list[idx] * weights[name]
    binary_preds_w = (w_scores >= best_thresholds['Weighted Ensemble']).astype(int)
    bootstrap_results['ROC AUC']['Weighted Ensemble'].append(roc_auc_score(y_true_boot, w_scores))
    bootstrap_results['PR AUC']['Weighted Ensemble'].append(average_precision_score(y_true_boot, w_scores))
    bootstrap_results['F1-Score']['Weighted Ensemble'].append(f1_score(y_true_boot, binary_preds_w))

    # Prepare Input for Meta-Learners (Stacking)
    X_meta_boot = np.column_stack(boot_probs_list)

    # 3. Stacked (LR)
    slr_scores = meta_lr.predict_proba(X_meta_boot)[:, 1]
    binary_preds_slr = (slr_scores >= best_thresholds['Stacked (LR)']).astype(int)
    bootstrap_results['ROC AUC']['Stacked (LR)'].append(roc_auc_score(y_true_boot, slr_scores))
    bootstrap_results['PR AUC']['Stacked (LR)'].append(average_precision_score(y_true_boot, slr_scores))
    bootstrap_results['F1-Score']['Stacked (LR)'].append(f1_score(y_true_boot, binary_preds_slr))

    # 4. Stacked (XGB)
    sxgb_scores = meta_xgb.predict_proba(X_meta_boot)[:, 1]
    binary_preds_sxgb = (sxgb_scores >= best_thresholds['Stacked (XGB)']).astype(int)
    bootstrap_results['ROC AUC']['Stacked (XGB)'].append(roc_auc_score(y_true_boot, sxgb_scores))
    bootstrap_results['PR AUC']['Stacked (XGB)'].append(average_precision_score(y_true_boot, sxgb_scores))
    bootstrap_results['F1-Score']['Stacked (XGB)'].append(f1_score(y_true_boot, binary_preds_sxgb))


# --- HOLM-BONFERRONI CORRECTION ---
print("\nCalculating P-Values & Applying Holm-Bonferroni Correction...")

p_vals_roc = []
p_vals_pr = []
p_vals_f1 = [] # Added for F1-Score
models_tested = []

ref_model_name = 'Averaged Ensemble' # Use the best ensemble as reference
ref_roc = bootstrap_results['ROC AUC'][ref_model_name]
ref_pr = bootstrap_results['PR AUC'][ref_model_name]
ref_f1 = bootstrap_results['F1-Score'][ref_model_name] # Added reference F1

# 1. Calculate Raw P-Values (Wilcoxon)
# Iterate through all models except the reference model itself
for name in model_names:
    if name == ref_model_name: continue

    # Ensure lengths are matched if any bootstrap iteration was skipped
    min_len_ref = len(ref_roc)
    min_len_current = min([len(bootstrap_results['ROC AUC'][name]), len(bootstrap_results['PR AUC'][name]), len(bootstrap_results['F1-Score'][name])])
    current_len = min(min_len_ref, min_len_current)

    # Skip if not enough samples for comparison
    if current_len < 2: continue # Wilcoxon needs at least 2 samples

    # Get truncated data for Wilcoxon test
    current_roc_data = bootstrap_results['ROC AUC'][name][:current_len]
    current_pr_data = bootstrap_results['PR AUC'][name][:current_len]
    current_f1_data = bootstrap_results['F1-Score'][name][:current_len]

    _, p_r = wilcoxon(current_roc_data, ref_roc[:current_len])
    _, p_p = wilcoxon(current_pr_data, ref_pr[:current_len])
    _, p_f = wilcoxon(current_f1_data, ref_f1[:current_len]) # Wilcoxon for F1

    p_vals_roc.append(p_r)
    p_vals_pr.append(p_p)
    p_vals_f1.append(p_f) # Append F1 p-value
    models_tested.append(name)

# 2. Apply Correction (separately for ROC, PR, and F1 families)
reject_roc, p_corrected_roc, _, _ = multipletests(p_vals_roc, alpha=alpha, method='holm')
reject_pr, p_corrected_pr, _, _ = multipletests(p_vals_pr, alpha=alpha, method='holm')
reject_f1, p_corrected_f1, _, _ = multipletests(p_vals_f1, alpha=alpha, method='holm') # Correct F1 p-values

# Map back to models
significance_map = {}
for i, name in enumerate(models_tested):
    significance_map[name] = {
        'ROC': '*' if reject_roc[i] else 'ns',
        'PR': '*' if reject_pr[i] else 'ns',
        'F1': '*' if reject_f1[i] else 'ns', # Added F1 significance
        'p_adj_roc': p_corrected_roc[i],
        'p_adj_pr': p_corrected_pr[i],
        'p_adj_f1': p_corrected_f1[i] # Added adjusted F1 p-value
    }

# --- FINAL TABLE ---
print("\n" + "="*140)
print(f"{'Model':<25} | {'ROC AUC (95% CI)':<28} | {'PR AUC (95% CI)':<28} | {'F1-Score (95% CI)':<28} | {'Sig (vs Ens)':<12}") # Updated header
print("-" * 140)

def get_ci(data):
    mean = np.mean(data)
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)
    return f"{mean:.3f} [{lower:.3f}, {upper:.3f}]"

# Print Rows (ensure consistency in model_names order if changed)
# Re-sort model_names to match the order in models_tested and ref_model_name at the end
printable_model_names = sorted(model_names, key=lambda x: (x == ref_model_name, x))

for name in printable_model_names:
    if name == ref_model_name:
        # Print reference model at the end
        continue

    # Get current length to avoid errors if some lists are shorter
    current_len = len(bootstrap_results['ROC AUC'][name])
    if current_len == 0: # Skip if no data for this model
        continue

    row_roc = get_ci(bootstrap_results['ROC AUC'][name][:current_len])
    row_pr = get_ci(bootstrap_results['PR AUC'][name][:current_len])
    row_f1 = get_ci(bootstrap_results['F1-Score'][name][:current_len]) # Added F1 CI

    sig_roc = significance_map[name]['ROC']
    sig_pr = significance_map[name]['PR']
    sig_f1 = significance_map[name]['F1'] # Added F1 significance

    # Mark significance if ANY metric is significant
    final_sig = "*" if ('*' in [sig_roc, sig_pr, sig_f1]) else "ns"

    print(f"{name:<25} | {row_roc:<28} | {row_pr:<28} | {row_f1:<28} | {final_sig:<12}")

print("-" * 140)

# Print Reference model row
ref_roc_ci = get_ci(bootstrap_results['ROC AUC'][ref_model_name])
ref_pr_ci = get_ci(bootstrap_results['PR AUC'][ref_model_name])
ref_f1_ci = get_ci(bootstrap_results['F1-Score'][ref_model_name])
print(f"{ref_model_name:<25} | {ref_roc_ci:<28} | {ref_pr_ci:<28} | {ref_f1_ci:<28} | {'Reference':<12}")

print("="*140)
print(f"* Statistically significant difference from {ref_model_name} (Holm-Bonferroni adj. p < {alpha}) ")

In [ ]:
import numpy as np
from scipy.stats import wilcoxon
from statsmodels.stats.multitest import multipletests

# --- 1. CONFIGURATION ---
alpha = 0.05
ref_model = 'Averaged Ensemble' # Reference for comparison

# Ensure model_names matches the keys in bootstrap_results
if 'model_names' not in globals():
    keys = list(bootstrap_results['ROC AUC'].keys())
    model_names = sorted(keys, key=lambda x: (x == ref_model, 'Ensemble' in x, x))

# --- 2. DIAGNOSE & FIX MISMATCH ---
print("Diagnosing Data Lengths...")
min_len = 999999999

# Find the shortest list length (in case of bootstrap interruptions)
for name in model_names:
    l = len(bootstrap_results['ROC AUC'][name]) # Use ROC AUC as a reference for length
    if l < min_len:
        min_len = l

print(f"Truncating all results to {min_len} samples to ensure alignment...")

# Truncate lists to match the minimum length for all metrics
for name in model_names:
    bootstrap_results['ROC AUC'][name] = bootstrap_results['ROC AUC'][name][:min_len]
    bootstrap_results['PR AUC'][name] = bootstrap_results['PR AUC'][name][:min_len]
    bootstrap_results['F1-Score'][name] = bootstrap_results['F1-Score'][name][:min_len]

# --- 3. CALCULATE RAW WILCOXON P-VALUES ---
print(f"Calculating raw p-values (Reference: {ref_model})...")
ref_roc = bootstrap_results['ROC AUC'][ref_model]
ref_pr = bootstrap_results['PR AUC'][ref_model]
ref_f1 = bootstrap_results['F1-Score'][ref_model]

p_vals_roc = []
p_vals_pr = []
p_vals_f1 = []
models_to_correct = []

for name in model_names:
    if name == ref_model: continue

    # Calculate Wilcoxon against Reference for each metric
    stat_roc, p_roc = wilcoxon(bootstrap_results['ROC AUC'][name], ref_roc)
    stat_pr, p_pr = wilcoxon(bootstrap_results['PR AUC'][name], ref_pr)
    stat_f1, p_f1 = wilcoxon(bootstrap_results['F1-Score'][name], ref_f1)

    p_vals_roc.append(p_roc)
    p_vals_pr.append(p_pr)
    p_vals_f1.append(p_f1)
    models_to_correct.append(name)

# --- 4. APPLY HOLM-BONFERRONI CORRECTION ---
reject_roc, p_corrected_roc, _, _ = multipletests(p_vals_roc, alpha=alpha, method='holm')
reject_pr, p_corrected_pr, _, _ = multipletests(p_vals_pr, alpha=alpha, method='holm')
reject_f1, p_corrected_f1, _, _ = multipletests(p_vals_f1, alpha=alpha, method='holm')

# Map results back for printing
adj_results = {}
for i, name in enumerate(models_to_correct):
    adj_results[name] = {
        'p_adj_roc': p_corrected_roc[i],
        'sig_roc': reject_roc[i],
        'p_adj_pr': p_corrected_pr[i],
        'sig_pr': reject_pr[i],
        'p_adj_f1': p_corrected_f1[i],
        'sig_f1': reject_f1[i]
    }

# --- 5. PRINT FINAL MANUSCRIPT TABLE ---
print("\n" + "="*170)
print(f"{'Model':<25} | {'ROC AUC (95% CI)':<28} | {'P-Value (Adj)':<15} | {'PR AUC (95% CI)':<28} | {'P-Value (Adj)':<15} | {'F1-Score (95% CI)':<28} | {'P-Value (Adj)':<15}")
print("-" * 170)

def get_ci(data):
    mean = np.mean(data)
    lower = np.percentile(data, 2.5)
    upper = np.percentile(data, 97.5)
    return f"{mean:.3f} [{lower:.3f}, {upper:.3f}]"

# Print Rows
for name in model_names:
    if name == ref_model: continue

    ci_roc_str = get_ci(bootstrap_results['ROC AUC'][name])
    ci_pr_str = get_ci(bootstrap_results['PR AUC'][name])
    ci_f1_str = get_ci(bootstrap_results['F1-Score'][name])

    # Retrieve stats
    stats = adj_results[name]

    p_adj_roc = stats['p_adj_roc']
    is_sig_roc = stats['sig_roc']

    p_adj_pr = stats['p_adj_pr']
    is_sig_pr = stats['sig_pr']

    p_adj_f1 = stats['p_adj_f1']
    is_sig_f1 = stats['sig_f1']

    p_roc_str = "< 0.001*" if p_adj_roc < 0.001 else f"{p_adj_roc:.3f}" + ("*" if is_sig_roc else "")
    p_pr_str = "< 0.001*" if p_adj_pr < 0.001 else f"{p_adj_pr:.3f}" + ("*" if is_sig_pr else "")
    p_f1_str = "< 0.001*" if p_adj_f1 < 0.001 else f"{p_adj_f1:.3f}" + ("*" if is_sig_f1 else "")

    print(f"{name:<25} | {ci_roc_str:<28} | {p_roc_str:<15} | {ci_pr_str:<28} | {p_pr_str:<15} | {ci_f1_str:<28} | {p_f1_str:<15}")

print("-" * 170)
# Print Reference at the bottom
ens_roc_ci = get_ci(bootstrap_results['ROC AUC'][ref_model])
ens_pr_ci = get_ci(bootstrap_results['PR AUC'][ref_model])
ens_f1_ci = get_ci(bootstrap_results['F1-Score'][ref_model])
print(f"{ref_model:<25} | {ens_roc_ci:<28} | {'Reference':<15} | {ens_pr_ci:<28} | {'Reference':<15} | {ens_f1_ci:<28} | {'Reference':<15}")
print("="*170)
print(f"* Statistically significant difference from {ref_model} (Holm-Bonferroni adjusted p < {alpha})")

In [ ]:
# @title 12. Generate Final "Winning" ROC Plot (Figure 3b)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score

# Setup
plt.figure(figsize=(10, 8))
plt.rcParams.update({'font.size': 12, 'font.family': 'serif'})

# 1. Baseline
fpr_ae, tpr_ae, _ = roc_curve(y_test, test_mae)
auc_ae = roc_auc_score(y_test, test_mae)
plt.plot(fpr_ae, tpr_ae, label=f'LSTM-AE Baseline (AUC = {auc_ae:.3f})',
         linestyle='--', color='gray', linewidth=2, alpha=0.5)

# 2. Key Individual Models (RF & XGB)
# We won't plot ALL of them to keep it clean, just the best ones + Winner
rf_probs = best_models['Random Forest'].predict_proba(X_test_feat)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)
auc_rf = roc_auc_score(y_test, rf_probs)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.3f})',
         color='#ff7f0e', linewidth=2, alpha=0.8)

# 3. Averaged Ensemble (Reference)
fpr_avg, tpr_avg, _ = roc_curve(y_test, test_probs_dict['Averaged Ensemble'])
auc_avg = roc_auc_score(y_test, test_probs_dict['Averaged Ensemble'])
plt.plot(fpr_avg, tpr_avg, label=f'Averaged Ensemble (AUC = {auc_avg:.3f})',
         color='blue', linewidth=2, linestyle='-.', alpha=0.8)

# 4. THE WINNER: Stacked (LR)
fpr_stack, tpr_stack, _ = roc_curve(y_test, test_probs_dict['Stacked (LR)'])
auc_stack = roc_auc_score(y_test, test_probs_dict['Stacked (LR)'])
plt.plot(fpr_stack, tpr_stack, label=f'Stacked Ensemble [LR] (AUC = {auc_stack:.3f})',
         color='black', linewidth=3.5, zorder=10)

# Formatting
plt.plot([0, 1], [0, 1], 'k:', alpha=0.4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.02])
plt.xlabel('False Positive Rate', fontweight='bold')
plt.ylabel('True Positive Rate', fontweight='bold')
plt.title('ROC Curves: Stacked Ensemble Optimization', fontweight='bold')
plt.legend(loc="lower right", frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Figure3b_Final_Stacked_Winner.pdf', dpi=600)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

print("--- DETAILED CLASSIFICATION REPORTS ---")
print("Note: For supervised models, the F1-score is calculated using the optimal threshold tuned on the validation set.\n      For ensembles, a threshold of 0.5 is used for simplicity unless a specific ensemble threshold was previously tuned for F1-score.\n      For One-Class SVM, its inherent -1/1 prediction is mapped to 1/0.\n")

# Combine all models and ensembles for reporting
all_models_for_report = {}
all_models_for_report.update(best_models)

# Add ensemble predictions to the dictionary for consistent processing
# Ensure these are aligned with how they were generated and stored in test_probs_dict
if 'Averaged Ensemble' in test_probs_dict:
    all_models_for_report['Averaged Ensemble'] = test_probs_dict['Averaged Ensemble']
if 'Weighted Ensemble' in test_probs_dict:
    all_models_for_report['Weighted Ensemble'] = test_probs_dict['Weighted Ensemble']
if 'Stacked (LR)' in test_probs_dict:
    all_models_for_report['Stacked (LR)'] = test_probs_dict['Stacked (LR)']
if 'Stacked (XGB)' in test_probs_dict:
    all_models_for_report['Stacked (XGB)'] = test_probs_dict['Stacked (XGB)']

for name, model_or_probs in all_models_for_report.items():
    print(f"\n{'='*50}\nCLASSIFICATION REPORT FOR: {name}\n{'='*50}")

    if name == "One-Class SVM":
        # OCSVM Logic: predict returns -1 for anomalies, 1 for normal
        # Map to 1 for positive class (dysbiosis), 0 for negative class (normal)
        preds = model_or_probs.predict(X_test_feat)
        binary_preds = np.where(preds == -1, 1, 0)
    elif isinstance(model_or_probs, np.ndarray): # This is for ensemble probabilities
        # For ensembles, we use 0.5 as the threshold for binary classification
        binary_preds = (model_or_probs >= 0.5).astype(int)
    else:
        # For other supervised models, use predict_proba and apply the tuned threshold
        probs = model_or_probs.predict_proba(X_test_feat)[:, 1]
        # Retrieve the best threshold for this specific model
        thresh = best_thresholds.get(name, 0.5) # Default to 0.5 if threshold not found (shouldn't happen)
        binary_preds = (probs >= thresh).astype(int)

    print(classification_report(y_test, binary_preds, target_names=['Class 0 (Normal)', 'Class 1 (Dysbiotic)']))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score
import numpy as np

print("--- GENERATING CONFUSION MATRICES ---")

# Combine trained models and ensemble probabilities into one dictionary
all_models_for_report = {}
all_models_for_report.update(best_models) # Your supervised models (RF, SVM, etc.)

# Add specific ensembles from your probability dictionary
# Ensure keys match those in 'best_thresholds' if you tuned them
ensemble_keys_for_cm = ['Averaged Ensemble', 'Weighted Ensemble', 'Stacked (LR)', 'Stacked (XGB)']
for key in ensemble_keys_for_cm:
    if key in test_probs_dict:
        all_models_for_report[key] = test_probs_dict[key]

for name, model_or_probs in all_models_for_report.items():
    print(f"\n{'='*50}\nCONFUSION MATRIX FOR: {name}\n{'='*50}")

    if name == "One-Class SVM":
        preds = model_or_probs.predict(X_test_feat)
        binary_preds = np.where(preds == -1, 1, 0) # Map OCSVM output to 0/1
    elif isinstance(model_or_probs, np.ndarray): # Ensembles are probabilities (numpy array)
        binary_preds = (model_or_probs >= 0.5).astype(int) # Default to 0.5 threshold for ensembles
    else:
        probs = model_or_probs.predict_proba(X_test_feat)[:, 1]
        thresh = best_thresholds.get(name, 0.5) # Use tuned threshold for supervised models
        binary_preds = (probs >= thresh).astype(int)

    # Calculate Confusion Matrix
    cm = confusion_matrix(y_test, binary_preds)

    # Calculate F1-score
    f1 = f1_score(y_test, binary_preds)

    # Plot Confusion Matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Predicted Normal', 'Predicted Dysbiotic'],
                yticklabels=['Actual Normal', 'Actual Dysbiotic'])
    plt.title(f'Confusion Matrix: {name}\nF1-Score: {f1:.4f}', fontsize=14)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)
    plt.tight_layout()
    plt.show()

In [ ]:
# @title 13. Baseline Comparison: Alpha & Beta Diversity (Corrected)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy, mannwhitneyu
from scipy.spatial.distance import braycurtis
from sklearn.metrics import roc_auc_score

# --- 1. PREPARE DATA ---
# FIX: Re-define X_train_normal just in case it is missing
X_train_normal = X_train[y_train == 0]

print(f"Normal Training Samples: {X_train_normal.shape[0]}")

# We use the mean abundance over the 14-day window as the representative profile
# Axis 1 is the time dimension (14 days)
X_train_mean = np.mean(X_train_normal, axis=1) # Shape: (N_train_normal, Features)
X_test_mean  = np.mean(X_test, axis=1)         # Shape: (N_test, Features)

# --- 2. ALPHA DIVERSITY (Shannon Index) ---
# H = -sum(p * log(p))
def calc_shannon(data):
    # Add epsilon to avoid log(0)
    return entropy(data + 1e-10, axis=1)

shannon_train = calc_shannon(X_train_mean)
shannon_test  = calc_shannon(X_test_mean)

# --- 3. BETA DIVERSITY (Distance to Healthy Centroid) ---
# Calculate the "Average Healthy Profile" (Centroid) from Training Data
healthy_centroid = np.mean(X_train_mean, axis=0)

# Calculate Bray-Curtis distance of every test sample to this centroid
beta_dist = []
for sample in X_test_mean:
    # Bray-Curtis: sum(|u-v|) / sum(|u+v|)
    d = braycurtis(sample, healthy_centroid)
    beta_dist.append(d)
beta_dist = np.array(beta_dist)

# --- 4. EVALUATION (Can these metrics predict Dysbiosis?) ---
# Note: Lower Shannon = Dysbiosis (usually), so we flip sign for AUC calculation
# (We want "Higher Score = Anomaly")
auc_alpha = roc_auc_score(y_test, -shannon_test)

# Higher Distance = Dysbiosis, so we use as is
auc_beta = roc_auc_score(y_test, beta_dist)

print(f"\n--- DIVERSITY METRIC RESULTS ---")
print(f"Alpha Diversity (Shannon) AUC:        {auc_alpha:.4f}")
print(f"Beta Diversity (Dist to Healthy) AUC: {auc_beta:.4f}")
print(f"DynaBiome (Random Forest) AUC:        0.8903") # Reference

# --- 5. STATISTICAL TEST ---
# Compare Normal vs Dysbiotic in Test Set
norm_idx = (y_test == 0)
dys_idx  = (y_test == 1)

stat_a, p_a = mannwhitneyu(shannon_test[norm_idx], shannon_test[dys_idx])
stat_b, p_b = mannwhitneyu(beta_dist[norm_idx], beta_dist[dys_idx])

print(f"\nStats (Normal vs. Dysbiotic):")
print(f"Alpha P-value: {p_a:.2e}")
print(f"Beta P-value:  {p_b:.2e}")

# --- 6. PLOT ---
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
plt.rcParams.update({'font.size': 12, 'font.family': 'serif'})

# Alpha Boxplot
sns.boxplot(x=y_test, y=shannon_test, ax=axes[0], palette=['#2ca02c', '#d62728'], hue=y_test, legend=False)
axes[0].set_xticks([0, 1]) # Explicitly set ticks before labels
axes[0].set_xticklabels(['Normal', 'Dysbiosis'])
axes[0].set_title(f'Alpha Diversity (Shannon)\nAUC = {auc_alpha:.2f}')
axes[0].set_ylabel('Shannon Index (Higher is Healthier)')
axes[0].grid(True, alpha=0.3)

# Beta Boxplot
sns.boxplot(x=y_test, y=beta_dist, ax=axes[1], palette=['#2ca02c', '#d62728'], hue=y_test, legend=False)
axes[1].set_xticks([0, 1]) # Explicitly set ticks before labels
axes[1].set_xticklabels(['Normal', 'Dysbiosis'])
axes[1].set_title(f'Beta Distance to Healthy Centroid\nAUC = {auc_beta:.2f}')
axes[1].set_ylabel('Bray-Curtis Distance (Higher is Anomalous)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Supplementary_Figure_S2_Diversity.pdf', dpi=600)
plt.show()

In [ ]:
# @title 14. Baseline Check: Median Predictor vs. LSTM
import numpy as np

# 1. Calculate the Median of the Training Data (Per Feature)
# We flatten the time dimension to get the median abundance of each genus across all history
# X_train shape: (N_samples, 14, N_features)
X_train_flat = X_train.reshape(-1, X_train.shape[2])
medians = np.median(X_train_flat, axis=0) # Shape: (N_features,)

# 2. Create "Dummy Predictions" for the Test Set
# We predict this static median for every timestep in the test set
X_test_flat = X_test.reshape(-1, X_test.shape[2])
# Tile the medians to match Test Set shape
dummy_preds = np.tile(medians, (X_test_flat.shape[0], 1))

# 3. Calculate MAE of the Median Baseline
median_baseline_mae = np.mean(np.abs(X_test_flat - dummy_preds))

# 4. Compare with LSTM Test MAE (calculated in Step 6)
lstm_mae_mean = np.mean(test_mae) # test_mae is vector of MAEs, take mean

print("--- CONVERGENCE CHECK ---")
print(f"Median Baseline MAE (The 'Lazy' Guess): {median_baseline_mae:.6f}")
print(f"DynaBiome LSTM MAE (Your Model):        {lstm_mae_mean:.6f}")

# Calculate Improvement Factor
improvement = median_baseline_mae / lstm_mae_mean
print(f"Factor of Improvement: {improvement:.1f}x lower error than baseline")